## SQL to NL to SQL architecture. 

In many cases, a company has already many anlytic assets such like SQLs, procedure, OLAP cubes and others in their firm. 

SQL is a valuable tool, but it can be challenging to use in a reusable way because it is not always clear what the query is doing.

For example, 

If someone want to know 'which department has poor performace than the average ?'

The goal is very simple, but its implementation SQL is very complex. 

There is a chasm between the business goal and the specific SQL implentation. 

In this case, pre-defined complex SQL can help to solve this problem. And we should suggest how to retrieve appropriate SQL from the lots of pre-defined SQLs.

In [1]:
# TODO: Implment this function
import pandas as pd

def crawl_prefined_sqls():
  sqls = []
  # goal = "SQL to check the delay time(second) before shipping for each order and product"
  sqls.append("""
CREATE TEMP FUNCTION convertIntervalToSecond(start_ts TIMESTAMP, end_ts TIMESTAMP)
RETURNS INT64
AS (
  (EXTRACT(DAY FROM (end_ts - start_ts)) * 86400 +
  EXTRACT(HOUR FROM (end_ts - start_ts)) * 3600 +
  EXTRACT(MINUTE FROM (end_ts - start_ts)) * 60 +
  EXTRACT(SECOND FROM (end_ts - start_ts)))
);
select convertIntervalToSecond(a.created_at, a.shipped_at) as order_pending_second, 
  convertIntervalToSecond(b.created_at, b.shipped_at) as product_pending_second, 
  a.status, a.order_id, a.user_id, a.gender, a.created_at, a.shipped_at, 
  b.product_id, b.created_at, b.shipped_at, b.delivered_at, b.returned_at
 from `bigquery-public-data.thelook_ecommerce.orders` a
 join `bigquery-public-data.thelook_ecommerce.order_items` b on (a.order_id = b.order_id)
 where a.status not in ('Cancelled') 
   and a.created_at between '2022-01-01' and '2022-06-30'
order by a.order_id, b.product_id
  """)
  return sqls


In [2]:
import vertexai
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
import os

PROJECT_ID = os.getenv("PROJECT_ID")  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

llm_vertex = VertexAI(
    #model_name="text-bison@latest",
    model_name="text-bison-32k",
    max_output_tokens=8000,
    temperature=0,
    top_p=0.8,
    top_k=40,
)

llm = llm_vertex

In [3]:
import json

def parse_json_response(llm_json_response) -> any:
  #print('llm response:'+ response)
  start_char = '['
  end_char = ']'
  if llm_json_response.find('[') == -1 or max(0,llm_json_response.find('{')) < llm_json_response.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_json_response.find(start_char)
  end_index = llm_json_response.rfind(end_char)
  json_data = llm_json_response[start_index:end_index+1]
  parsed_json = json.loads(json_data)
  return parsed_json


In [4]:
import ast

def parse_python_object(llm_python_object) -> any:
  print('llm response:'+ llm_python_object)
  if llm_python_object.find('{') == -1:
    start_char = '['
    end_char = ']'
  elif llm_python_object.find('[') == -1 or llm_python_object.find('{') < llm_python_object.find('[') :
    start_char = '{'
    end_char = '}'
  start_index = llm_python_object.find(start_char)
  end_index = llm_python_object.rfind(end_char)
  object_data = llm_python_object[start_index:end_index+1]
  print(object_data)
  parsed_object = ast.literal_eval(object_data)
  return parsed_object


In [59]:

def explain_sql(sql):
  example_json = """
  {
    "business_goal": "explanation of the SQL",
    "prepared_statement": "select convertIntervalToSecond(a.created_at, a.shipped_at) as order_pending_second from sample_table where created_at between ? and ?",
    "filter_columns": [
      {
        "table_name": "sample_table",
        "column_name": "created_at",
        "business goal": "filter by order created date",
        "operator": "between",
        "column_type" : "TIMESTAMP",
        "filter names": ["order_created_at_start","order_created_at_end"]
        "fitler_order": 1
      }
    ],
  }
  """
  prompt_template = """You are a serverside developer. 
  Please convert SQL to prepared statement and extract filters from the SQL in a json format. filter_columns include only predicate columns. Do not suggest python code.


  target sql:
  {sql}

  ----------------------------

  output format(example): json
  {example_json}
  """
  prompt = prompt_template.format(sql=sql, example_json=example_json)
  response = llm.predict(prompt)
  print(response)
  return parse_json_response(response)

In [60]:

assetized_queries = []

## In some cases, the LLM model can't extract the filter columns properly.

for sql in crawl_prefined_sqls():
  assetized_queries.append(explain_sql(sql))
  

 ```json
{
    "business_goal": "Get order pending time and product pending time for each order",
    "prepared_statement": "CREATE TEMP FUNCTION convertIntervalToSecond(start_ts TIMESTAMP, end_ts TIMESTAMP)\nRETURNS INT64\nAS (\n  (EXTRACT(DAY FROM (end_ts - start_ts)) * 86400 +\n  EXTRACT(HOUR FROM (end_ts - start_ts)) * 3600 +\n  EXTRACT(MINUTE FROM (end_ts - start_ts)) * 60 +\n  EXTRACT(SECOND FROM (end_ts - start_ts)))\n);\nselect convertIntervalToSecond(a.created_at, a.shipped_at) as order_pending_second, \n  convertIntervalToSecond(b.created_at, b.shipped_at) as product_pending_second, \n  a.status, a.order_id, a.user_id, a.gender, a.created_at, a.shipped_at, \n  b.product_id, b.created_at, b.shipped_at, b.delivered_at, b.returned_at\n from `bigquery-public-data.thelook_ecommerce.orders` a\n join `bigquery-public-data.thelook_ecommerce.order_items` b on (a.order_id = b.order_id)\n where a.status not in (?) \n   and a.created_at between ? and ?\norder by a.order_id, b.product_id"

In [61]:
from vector_util import VectorDatabase

vdb = VectorDatabase()

vdb.truncate_table()

In [62]:
from langchain.embeddings import VertexAIEmbeddings

embeddings = VertexAIEmbeddings()

In [64]:
def write_goal_to_vdb(assetized_queries):
  for assetized_query in assetized_queries:
    description = assetized_query['business_goal']
    sql = assetized_query['prepared_statement']
    parameters = assetized_query['filter_columns']
    desc_vector = embeddings.embed_query(description)
    vdb.insert_record(sql=sql, parameters=parameters, description=description, explore_view=None, model_name=None, table_name=None, column_schema=None, desc_vector=desc_vector)

write_goal_to_vdb(assetized_queries)

In [65]:
def get_related_query(question):
  test_embedding =  embeddings.embed_query(question)
  result = None
  with vdb.get_connection() as conn:
    try:
      with conn.cursor() as cur:
        select_record = (str(test_embedding).replace(' ',''),)
        cur.execute(f"SELECT sql, parameters, description FROM rag_test where (1 - (desc_vector <=> %s)) > 0.6 limit 1", select_record)
        if cur.rowcount == 0:
          return None
        rs = cur.fetchone()
        result = {
          'prepared_statement': rs[0],
          'filter_columns': rs[1],
          'description': rs[2]
        }
        print(result)
    except Exception as e:
      print(e)
  return result


In [66]:
question = "How to check the delay time(second) before shipping for each order and product?"

assetized_query = get_related_query(question)
print(assetized_query)


{'prepared_statement': 'CREATE TEMP FUNCTION convertIntervalToSecond(start_ts TIMESTAMP, end_ts TIMESTAMP)\nRETURNS INT64\nAS (\n  (EXTRACT(DAY FROM (end_ts - start_ts)) * 86400 +\n  EXTRACT(HOUR FROM (end_ts - start_ts)) * 3600 +\n  EXTRACT(MINUTE FROM (end_ts - start_ts)) * 60 +\n  EXTRACT(SECOND FROM (end_ts - start_ts)))\n);\nselect convertIntervalToSecond(a.created_at, a.shipped_at) as order_pending_second, \n  convertIntervalToSecond(b.created_at, b.shipped_at) as product_pending_second, \n  a.status, a.order_id, a.user_id, a.gender, a.created_at, a.shipped_at, \n  b.product_id, b.created_at, b.shipped_at, b.delivered_at, b.returned_at\n from `bigquery-public-data.thelook_ecommerce.orders` a\n join `bigquery-public-data.thelook_ecommerce.order_items` b on (a.order_id = b.order_id)\n where a.status not in (?) \n   and a.created_at between ? and ?\norder by a.order_id, b.product_id', 'filter_columns': "[{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`', 'column_name'

In [81]:

def extract_filter_values(assetized_query, question):
  example_json = """
  {
    "filter_columns": [
      {
        "table_name": "sample_table",
        "column_name": "col_1",
        "operator" : "in", 
        "column_type" : "STRING",
        "filter_names": ["col_1_filter"],
        "filter_values": [null],
        "filter_order": 1
      },
      {
        "table_name": "sample_table",
        "column_name": "col_2",
        "operator" : "=", 
        "column_type" : "STRING",
        "filter_names": ["col_2_filter"],
        "filter_values": ["2022-01"],
        "filter_order": 2
      }
    ]
  }
  """

  prompt_template = """You are a serverside developer. Extract filter values from the question and fill the values into the 'filter_values' item for the given filter columns. Do not suggest codes. Output should be the following JSON format.

  given question:
  {question}
  
  given sql:
  {sql}

  given filters:
  {filters}

  ----------------------------

  output format(example) : json
  {example_json}

  """
  sql = assetized_query['prepared_statement']
  filters = assetized_query['filter_columns']
  prompt = prompt_template.format(sql=sql, filters=filters, example_json=example_json, question=question)
  response = llm.predict(prompt)
  return response


In [82]:
question_enriched = "How to check the delay time(second) before shipping for each order and product in this year(2023)?"

response = extract_filter_values(assetized_query, question_enriched)

In [83]:
response

' {\n    "filter_columns": [\n      {\n        "table_name": "`bigquery-public-data.thelook_ecommerce.orders`",\n        "column_name": "status",\n        "operator": "not in",\n        "column_type": "STRING",\n        "filter_names": [\n         "order_status"\n        ],\n        "filter_values": [],\n        "filter_order": 1\n      },\n      {\n        "table_name": "`bigquery-public-data.thelook_ecommerce.orders`",\n        "column_name": "created_at",\n        "operator": "between",\n        "column_type": "TIMESTAMP",\n        "filter_names": [\n         "order_created_at_start",\n         "order_created_at_end"\n        ],\n        "filter_values": [\n         "2023-01-01",\n         "2023-12-31"\n        ],\n        "filter_order": 2\n      }\n    ]\n  }'

In [84]:
filter_values = parse_json_response(response)

In [85]:
filter_values

{'filter_columns': [{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'status',
   'operator': 'not in',
   'column_type': 'STRING',
   'filter_names': ['order_status'],
   'filter_values': [],
   'filter_order': 1},
  {'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'created_at',
   'operator': 'between',
   'column_type': 'TIMESTAMP',
   'filter_names': ['order_created_at_start', 'order_created_at_end'],
   'filter_values': ['2023-01-01', '2023-12-31'],
   'filter_order': 2}]}

In [87]:

def check_unfilled_filters(filter_values):
  unfilled_filters = []
  for parameter in filter_values['filter_columns']:
    if None in parameter['filter_values'] or len(parameter['filter_values']) == 0:
      unfilled_filters.append(parameter)
  return unfilled_filters



In [88]:
unfilled_filters = check_unfilled_filters(filter_values)

In [89]:
unfilled_filters

[{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
  'column_name': 'status',
  'operator': 'not in',
  'column_type': 'STRING',
  'filter_names': ['order_status'],
  'filter_values': [],
  'filter_order': 1}]

In [90]:

def make_response_to_fill_filter_values(unfilled_filters, question):
  example_json = """
  {
    "agent_response": "..."
  }
  """
  prompt_template = """You are an automatic agent to serve natural language to SQL conversion. Please guide the user to fill the missing filter values in the given question and unfilled filters. Output should be the following JSON format.

  question:
  {question}

  unfilled filters:
  {unfilled_filters}

  output format json:
  {example_json}
  """
  prompt = prompt_template.format(unfilled_filters=unfilled_filters, question=question, example_json=example_json)
  response = llm.predict(prompt)
  return response

In [91]:
response = make_response_to_fill_filter_values(unfilled_filters, question=question_enriched)


In [92]:
response

' {\n    "agent_response": "What is the order status that you want to exclude?"\n  }'

In [93]:
additional_response = "I want to know not cancelled orders only"

In [99]:

def extract_filter_values_with_additional_response(unfilled_filters, additional_response):
  example_json = """
  {
    "filter_columns": [
      {
        "column_name": "col_1",
        "operator" : "in", 
        "column_type" : "STRING",
        "filter_names": ["col_1_filter"],
        "filter_values": [null]
      }
    ]
  }
  """

  prompt_template = """You are a serverside developer. Extract filter values from the user response and unfilled filter information. Do not suggest codes. Output should be the following JSON format.

  output format json:
  {example_json}

  ----------------------------
  user response :
  {user_response}

  unfilled filters:
  {unfilled_filters}
  """
  prompt = prompt_template.format(unfilled_filters=unfilled_filters, user_response=additional_response, example_json=example_json)
  response = llm.predict(prompt)
  return response

In [100]:
additional_filter_values = parse_json_response(extract_filter_values_with_additional_response(unfilled_filters, additional_response))

In [101]:
additional_filter_values

{'filter_columns': [{'column_name': 'status',
   'operator': 'not in',
   'column_type': 'STRING',
   'filter_names': ['order_status'],
   'filter_values': ['cancelled']}]}

In [104]:
def merge_filter_values(filter_values, additional_filter_values):
  original_parameters = filter_values['filter_columns']
  additional_parameters = additional_filter_values['filter_columns']
  for org_parameter in original_parameters:
    for add_parameter in additional_parameters:
      if org_parameter['column_name'] == add_parameter['column_name']:
        org_parameter['filter_values'] = add_parameter['filter_values']
  return filter_values


In [105]:
merged_filter_values = merge_filter_values(filter_values, additional_filter_values)

In [106]:
merged_filter_values

{'filter_columns': [{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'status',
   'operator': 'not in',
   'column_type': 'STRING',
   'filter_names': ['order_status'],
   'filter_values': ['cancelled'],
   'filter_order': 1},
  {'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'created_at',
   'operator': 'between',
   'column_type': 'TIMESTAMP',
   'filter_names': ['order_created_at_start', 'order_created_at_end'],
   'filter_values': ['2023-01-01', '2023-12-31'],
   'filter_order': 2}]}

As you can see, LLM can handle extraction / tranformation and validation as well. 

Next step is very similar with 'Direction Conversion'.

In [107]:
from google.cloud import bigquery
client = bigquery.Client()

In [108]:
def get_field_unique_values(matched_table, matched_field):
  if matched_table[0] != '`' :
    matched_table = '`' + matched_table + '`'
  sql_query = f"with distinct_values as ( select distinct {matched_field} as {matched_field} from {matched_table} ) select {matched_field}, (select count(1) from distinct_values) as total_count from distinct_values limit 500"
  df = client.query(sql_query).to_dataframe()
  return df[matched_field].tolist(), df['total_count'][0]
  

In [122]:
import ast


def choose_right_filter_value(filter_values, wanted_value):
  prompt_template = """As a looker developer, choose right filter value for the wanted value below without changing filter value itself.

  filter_values : {filter_values}

  wanted_values: {wanted_value}

  answer format: python list
[filter_value1, filter_value2, ...]
  """
  response = llm.predict(prompt_template.format(filter_values=filter_values, wanted_value=wanted_value))
  return response 

def adjust_filter_value(filter_columns):
  for filter in filter_columns:
    matched_table = filter['table_name']
    matched_field = filter['column_name']
    filter['unique_values'], filter['unique_count'] = get_field_unique_values(matched_table, matched_field)
    # TODO: if unique_count < 500, then choose right filter value in the unique value list.
    if filter['unique_count'] < 500:
      response = choose_right_filter_value(filter['unique_values'], filter['filter_values'])
      if response.strip().find("```json") == 0 :
        filter['adjust_filter_values'] = parse_json_response(response)
      else:
        filter['adjust_filter_values'] = parse_python_object(response)
    else:
      filter['adjust_filter_values'] = filter['filter_values']
    filter['unique_values'] = None
  
  

In [123]:
adjust_filter_value(merged_filter_values['filter_columns'])

llm response: ```python
['Cancelled']
```
['Cancelled']


In [124]:
merged_filter_values

{'filter_columns': [{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'status',
   'operator': 'not in',
   'column_type': 'STRING',
   'filter_names': ['order_status'],
   'filter_values': ['cancelled'],
   'filter_order': 1,
   'unique_values': None,
   'unique_count': 5,
   'adjust_filter_values': ['Cancelled'],
   'distinct_values': None},
  {'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_name': 'created_at',
   'operator': 'between',
   'column_type': 'TIMESTAMP',
   'filter_names': ['order_created_at_start', 'order_created_at_end'],
   'filter_values': ['2023-01-01', '2023-12-31'],
   'filter_order': 2,
   'unique_values': None,
   'unique_count': 117439,
   'adjust_filter_values': ['2023-01-01', '2023-12-31'],
   'distinct_values': None}]}

In [134]:
def prepared_statement_with_filter_values_in_bigquery(sql_and_filters):
  prepared_statement = sql_and_filters['prepared_statement']
  query_parameters = []
  for filter_column in sql_and_filters['filter_columns']:
    if len(filter_column['adjust_filter_values']) > 1:
      if len(filter_column['filter_names']) > 1:
        for filter_value in filter_column['adjust_filter_values']:
          if(filter_column['column_type'] == 'FLOAT64'):
            query_parameters.append(bigquery.ScalarQueryParameter(None, "FLOAT64", filter_value))
          elif(filter_column['column_type'] == 'INT64'):
            query_parameters.append(bigquery.ScalarQueryParameter(None, "INT64", filter_value))
          else:
            query_parameters.append(bigquery.ScalarQueryParameter(None, "STRING", filter_value))  
      else:
        if(filter_column['column_type'] == 'FLOAT64'):
          query_parameters.append(bigquery.ArrayQueryParameter(None, "FLOAT64", filter_column['adjust_filter_values']))
        elif(filter_column['column_type'] == 'INT64'):
          query_parameters.append(bigquery.ArrayQueryParameter(None, "INT64", filter_column['adjust_filter_values']))
        else:
          query_parameters.append(bigquery.ArrayQueryParameter(None, "STRING", filter_column['adjust_filter_values']))
    else:
      if(filter_column['column_type'] == 'FLOAT64'):
        query_parameters.append(bigquery.ScalarQueryParameter(None, "FLOAT64", filter_column['adjust_filter_values'][0]))
      elif(filter_column['column_type'] == 'INT64'):
        query_parameters.append(bigquery.ScalarQueryParameter(None, "INT64", filter_column['adjust_filter_values'][0]))
      else:
        query_parameters.append(bigquery.ScalarQueryParameter(None, "STRING", filter_column['adjust_filter_values'][0]))  
  job_config = bigquery.QueryJobConfig(
    query_parameters=query_parameters
  )
  print(query_parameters)
  query_job = client.query(prepared_statement, job_config=job_config)
  return query_job.to_dataframe()

In [135]:
sql_and_filters = {
  'prepared_statement':assetized_query['prepared_statement'],
  'filter_columns': merged_filter_values['filter_columns']
}

In [136]:
sql_and_filters

{'prepared_statement': 'CREATE TEMP FUNCTION convertIntervalToSecond(start_ts TIMESTAMP, end_ts TIMESTAMP)\nRETURNS INT64\nAS (\n  (EXTRACT(DAY FROM (end_ts - start_ts)) * 86400 +\n  EXTRACT(HOUR FROM (end_ts - start_ts)) * 3600 +\n  EXTRACT(MINUTE FROM (end_ts - start_ts)) * 60 +\n  EXTRACT(SECOND FROM (end_ts - start_ts)))\n);\nselect convertIntervalToSecond(a.created_at, a.shipped_at) as order_pending_second, \n  convertIntervalToSecond(b.created_at, b.shipped_at) as product_pending_second, \n  a.status, a.order_id, a.user_id, a.gender, a.created_at, a.shipped_at, \n  b.product_id, b.created_at, b.shipped_at, b.delivered_at, b.returned_at\n from `bigquery-public-data.thelook_ecommerce.orders` a\n join `bigquery-public-data.thelook_ecommerce.order_items` b on (a.order_id = b.order_id)\n where a.status not in (?) \n   and a.created_at between ? and ?\norder by a.order_id, b.product_id',
 'filter_columns': [{'table_name': '`bigquery-public-data.thelook_ecommerce.orders`',
   'column_na

In [137]:
df_result = prepared_statement_with_filter_values_in_bigquery(sql_and_filters)

[ScalarQueryParameter(None, 'STRING', 'Cancelled'), ScalarQueryParameter(None, 'STRING', '2023-01-01'), ScalarQueryParameter(None, 'STRING', '2023-12-31')]


In [139]:
df_result.head(10)

,order_pending_second,product_pending_second,status,order_id,user_id,gender,created_at,shipped_at,product_id,created_at_1,shipped_at_1,delivered_at,returned_at
0,2520,15675,Shipped,2,3,F,2023-02-20 12:34:00+00:00,2023-02-20 13:16:00+00:00,13848,2023-02-20 08:54:45+00:00,2023-02-20 13:16:00+00:00,NaT,NaT
1,210660,217579,Returned,5,6,F,2023-11-06 16:57:00+00:00,2023-11-09 03:28:00+00:00,3735,2023-11-06 15:01:41+00:00,2023-11-09 03:28:00+00:00,2023-11-10 00:43:00+00:00,2023-11-11 01:32:00+00:00
2,76020,82580,Complete,6,7,M,2023-11-24 10:44:00+00:00,2023-11-25 07:51:00+00:00,28363,2023-11-24 08:54:40+00:00,2023-11-25 07:51:00+00:00,2023-11-25 23:11:00+00:00,NaT
3,<NA>,<NA>,Processing,7,7,M,2023-07-10 10:44:00+00:00,NaT,24678,2023-07-10 07:46:42+00:00,NaT,NaT,NaT
4,66480,72788,Complete,8,7,M,2023-05-04 10:44:00+00:00,2023-05-05 05:12:00+00:00,18588,2023-05-04 08:58:52+00:00,2023-05-05 05:12:00+00:00,2023-05-07 03:48:00+00:00,NaT
5,66480,-14648,Complete,8,7,M,2023-05-04 10:44:00+00:00,2023-05-05 05:12:00+00:00,27795,2023-05-05 09:16:08+00:00,2023-05-05 05:12:00+00:00,2023-05-07 03:48:00+00:00,NaT
6,<NA>,<NA>,Processing,9,7,M,2023-08-24 10:44:00+00:00,NaT,16479,2023-08-24 10:00:36+00:00,NaT,NaT,NaT
7,<NA>,<NA>,Processing,10,8,F,2023-01-07 14:21:00+00:00,NaT,6282,2023-01-08 12:54:13+00:00,NaT,NaT,NaT
8,<NA>,<NA>,Processing,10,8,F,2023-01-07 14:21:00+00:00,NaT,6972,2023-01-11 13:44:11+00:00,NaT,NaT,NaT
9,146640,160028,Complete,11,9,F,2023-08-01 08:45:00+00:00,2023-08-03 01:29:00+00:00,9652,2023-08-01 05:01:52+00:00,2023-08-03 01:29:00+00:00,2023-08-04 23:30:00+00:00,NaT
